In [ ]:
!mkdir efficientnet
!cp -r ../input/segmentation-masters/efficientnet-master/efficientnet-master/efficientnet/* ./efficientnet

In [ ]:
!mkdir classification_models
!cp -r ../input/segmentation-masters/classification_models-master/classification_models-master/classification_models/* ./classification_models

In [ ]:
!mkdir segmentation_models
!cp -r ../input/segmentation-masters/segmentation_models-master/segmentation_models-master/segmentation_models/* ./segmentation_models

In [ ]:
!ls ./efficientnet

As the competition does not allow commit with the kernel that uses internet connection, we use offline installation

In [ ]:
! python ../input/mlcomp/mlcomp/mlcomp/setup.py

### Import required libraries

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import os
import gc
import matplotlib.pyplot as plt

import numpy as np
import cv2
import albumentations as A
from tqdm import tqdm_notebook
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.jit import load

from mlcomp.contrib.transform.albumentations import ChannelTranspose
from mlcomp.contrib.dataset.classify import ImageDataset
from mlcomp.contrib.transform.rle import rle2mask, mask2rle
from mlcomp.contrib.transform.tta import TtaWrap

Catalyst allows to trace models. That is an extremely useful features in Pytorch since 1.0 version: 

https://pytorch.org/docs/stable/jit.html

Now we can load models without re-defining them

In [ ]:
unet_se_resnext50_32x4d = load('/kaggle/input/severstalmodels/unet_se_resnext50_32x4d.pth').cuda()
unet_mobilenet2 = load('/kaggle/input/severstalmodels/unet_mobilenet2.pth').cuda()
unet_resnet34 = load('/kaggle/input/severstalmodels/unet_resnet34.pth').cuda()

### Models' mean aggregator

In [ ]:
class Model:
    def __init__(self, models):
        self.models = models
    
    def __call__(self, x):
        res = []
        x = x.cuda()
        with torch.no_grad():
            for m in self.models:
                res.append(m(x))
        res = torch.stack(res)
        return torch.mean(res, dim=0)

model = Model([unet_se_resnext50_32x4d, unet_mobilenet2, unet_resnet34])

### Create TTA transforms, datasets, loaders

In [ ]:
def create_transforms(additional):
    res = list(additional)
    # add necessary transformations
    res.extend([
        A.Normalize(
            mean=(0.485, 0.456, 0.406), std=(0.230, 0.225, 0.223)
        ),
        ChannelTranspose()
    ])
    res = A.Compose(res)
    return res

img_folder = '/kaggle/input/severstal-steel-defect-detection/test_images'
batch_size = 2
num_workers = 0

# Different transforms for TTA wrapper
transforms = [
    [],
    [A.HorizontalFlip(p=1)]
]

transforms = [create_transforms(t) for t in transforms]
datasets = [TtaWrap(ImageDataset(img_folder=img_folder, transforms=t), tfms=t) for t in transforms]
loaders = [DataLoader(d, num_workers=num_workers, batch_size=batch_size, shuffle=False) for d in datasets]

### Loaders' mean aggregator

In [ ]:
thresholds = [0.5, 0.7, 0.5, 0.5]
min_area = [600, 600, 1000, 2000]

res = []
# Iterate over all TTA loaders
total = len(datasets[0])//batch_size
for loaders_batch in tqdm_notebook(zip(*loaders), total=total):
    preds = []
    image_file = []
    for i, batch in enumerate(loaders_batch):
        features = batch['features'].cuda()
        p = torch.sigmoid(model(features))
        # inverse operations for TTA
        p = datasets[i].inverse(p)
        preds.append(p)
        image_file = batch['image_file']
    
    # TTA mean
    preds = torch.stack(preds)
    preds = torch.mean(preds, dim=0)
    preds = preds.detach().cpu().numpy()
    
    # Batch post processing
    for p, file in zip(preds, image_file):
        file = os.path.basename(file)
        # Image postprocessing
        for i in range(4):
            p_channel = p[i]
            imageid_classid = file+'_'+str(i+1)
            p_channel = (p_channel>thresholds[i]).astype(np.uint8)
            if p_channel.sum() < min_area[i]:
                p_channel = np.zeros(p_channel.shape, dtype=p_channel.dtype)

            res.append({
                'ImageId_ClassId': imageid_classid,
                'EncodedPixels': mask2rle(p_channel)
            })
            
df = pd.DataFrame(res)
df = df.fillna('')   

Save predictions

In [ ]:
import matplotlib.pyplot as plt
import random
import math as math
import os
import numpy as np
import pandas as pd
import cv2
from PIL import Image
import os
import tensorflow as tf
import keras.backend as K
from keras.utils import Sequence
from keras.applications import Xception
from keras.layers import UpSampling2D, Conv2D, Activation, LeakyReLU, BatchNormalization
from keras import Model
from keras.losses import binary_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras.optimizers import Adam

import imgaug as ia
import imgaug.augmenters as iaa
import segmentation_models as sm

import efficientnet.keras as efn 
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten

In [ ]:
base = efn.EfficientNetB3(weights=None, include_top=False, input_shape=(256, 1600, 3), pooling='avg')
base.trainable=True
dropout_dense_layer = 0.2 # for B0

classifier_model = Sequential()
classifier_model.add(base)
classifier_model.add(Dropout(dropout_dense_layer))
classifier_model.add(Dense(4, activation='sigmoid'))

classifier_model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

classifier_model.load_weights('../input/single-models/B3_opt_multi_new3.h5')

IMAGE_RGB_MEAN = [0.485, 0.456, 0.406]
IMAGE_RGB_STD  = [0.229, 0.224, 0.225]

def normalize(images):
    images = images / 255.0
    images[:,:,:,0] = (images[:,:,:,0]-IMAGE_RGB_MEAN[0])/IMAGE_RGB_STD[0]
    images[:,:,:,1] = (images[:,:,:,1]-IMAGE_RGB_MEAN[1])/IMAGE_RGB_STD[1]
    images[:,:,:,2] = (images[:,:,:,2]-IMAGE_RGB_MEAN[2])/IMAGE_RGB_STD[2]
    return images
    
def denormalize(images):
    images[:,:,:,0] = images[:,:,:,0]*IMAGE_RGB_STD[0]+IMAGE_RGB_MEAN[0]
    images[:,:,:,1] = images[:,:,:,1]*IMAGE_RGB_STD[1]+IMAGE_RGB_MEAN[1]
    images[:,:,:,2] = images[:,:,:,2]*IMAGE_RGB_STD[2]+IMAGE_RGB_MEAN[2]
    images = np.array(images * 255).astype('uint8')
    
TEST_PATH = '../input/severstal-steel-defect-detection/test_images/'
def prepareData(source, path, cleanup=True):
    source['defect'] = False
    if cleanup: source.EncodedPixels = ''
    
    source['ClassId'] = source['ImageId_ClassId'].str[-1:]
    source['ImageId'] = source['ImageId_ClassId'].str[:-2]
    source = source[['ImageId_ClassId', 'ImageId','ClassId','defect','EncodedPixels']]
    source.ClassId = source.ClassId.astype('int')
    source['path'] = path + source['ImageId']
    return source

In [ ]:
test_df = pd.read_csv('../input/severstal-steel-defect-detection/sample_submission.csv')
test_df = prepareData(test_df, TEST_PATH)

# Predictions

ids = test_df['ImageId'].unique()
test_df.EncodedPixels = ''


class_weights = [0.5, 0.5, 0.5, 0.5] 

for picIdx in tqdm_notebook(range(len(ids))):
    
    batch = np.zeros((4, 256, 1600, 3))
    filename = ids[picIdx]  
    img = cv2.imread(TEST_PATH+filename)    

    batch[0, :, :, :] = img
    batch[1, :, :, :] = img[:,::-1,:]
    batch[2, :, :, :] = img[::-1,:,:]
    batch[3, :, :, :] = img[::-1,::-1,:]
    
    batch = normalize(batch)
    classTTA = classifier_model.predict(batch)
    hasDefect = np.mean(classTTA, axis=0)>class_weights

    for i, classId in enumerate(hasDefect):
        if classId == False:

            name = filename+"_"+str(i+1)
            line = test_df[test_df.ImageId_ClassId == name].index[0] 

            test_df.loc[line, 'EncodedPixels'] = -1

In [ ]:
mark = test_df[test_df.EncodedPixels == -1]
mark.ClassId.value_counts()

sub = df.copy()

In [ ]:
for filename in mark.ImageId_ClassId:
    sub.loc[sub.ImageId_ClassId == filename, 'EncodedPixels'] = -1

sub = sub.replace(-1, '')
sub.to_csv('submission.csv', index=False)
sub.head(20)

Histogram of predictions

In [ ]:
sub['Image'] = sub['ImageId_ClassId'].map(lambda x: x.split('_')[0])
sub['Class'] = sub['ImageId_ClassId'].map(lambda x: x.split('_')[1])
sub['empty'] = sub['EncodedPixels'].map(lambda x: not x)
sub[sub['empty'] == False]['Class'].value_counts()